In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import re
from lxml.html import fromstring
import pandas as pd
pd.set_option('display.max_colwidth', 500)
import random

In [88]:
search_keyword = 'rosa'
base_url = 'https://www.wardow.com'
search_path = '/catalogsearch/result/?q='

def construct_url(base, path, keyword): 
    return base + path + keyword

In [166]:
class Search_Results_Scraper: 
    def __init__(self, url): 
        self.url = url 
    def get_html(self): 
        try: 
            response = requests.get(self.url)
            response.raise_for_status()
        except requests.RequestException: 
            print('A problem occurred with your request')
            raise
        html = fromstring(response.content)
        return html
    def get_search_results(self): 
        results = []
        results_display = {}
        page = self.get_html()
        page_results = page.findall('.//div[@class="product-tile__info"]')    
        for item in page_results: 
            item_title = item.xpath('.//a/@title')[0]
            item_url = item.xpath('.//a/@href')[0]
            if search_keyword in item_title.lower(): 
                results.append(item_url)
                results_display[item_title] = item_url 
        next_page = base_url + page.xpath('//button[@class="button btn-subtle next"]/@value')[0]
        print('Your search term has found the following results:')
        print(pd.DataFrame.from_dict(results_display, orient='index', columns=['url']))
        try: 
            get_search_results(get_html(next_page))
        except: 
            print('No further results pages could be found...')
        return results 

In [176]:
search_results = Search_Results_Scraper(construct_url(base_url, search_path, search_keyword))
results = search_results.get_search_results()

Your search term has found the following results:
                                                                                            url
Harvest Label Norikura Rolltop Rucksack rosa  https://www.wardow.com/harvest-label-norikura-...
Bree PNCH 62 Kuriertasche rosa                https://www.wardow.com/bree-pnch-62-kuriertasc...
Lancaster Smooth Even Umhängetasche rosa      https://www.wardow.com/lancaster-smooth-even-u...
Lancaster Vendome Umhängetasche rosa          https://www.wardow.com/lancaster-vendome-umhan...
Bree PNCH 61 Kuriertasche rosa                https://www.wardow.com/bree-pnch-61-kuriertasc...
...                                                                                         ...
Titan Spotlight ZIP Rucksack rosa             https://www.wardow.com/titan-spotlight-zip-ruc...
Rains Msn Mini Rucksack rosa                  https://www.wardow.com/rains-msn-mini-rucksack...
DKNY Elissa Umhängetasche rosa                https://www.wardow.com/dkny-elissa-umhan

Your search term has found the following results:
                                                                                                    0
Jost Mesh X-Change (3in1) Bag M Rucksack-Tasche...  https://www.wardow.com/jost-mesh-x-change-3in1...
Doughnut Nature Pale Macaroon Mini Rucksack ros...  https://www.wardow.com/doughnut-nature-pale-ma...
Ucon Acrobatics Lotus Hajo Mini Rolltop Rucksac...  https://www.wardow.com/ucon-acrobatics-lotus-h...
The Bridge Vallombrosa Geldbörse beige              https://www.wardow.com/the-bridge-vallombrosa-...
Jost Mesh X-Change (3in1) Bag Mini Rucksack-Tas...  https://www.wardow.com/jost-mesh-x-change-3in1...
Eastpak Dee Rucksack rosa                           https://www.wardow.com/eastpak-dee-rucksack-ro...
Jost Farum X-Change (3in1) Bag Mini Rucksack-Ta...  https://www.wardow.com/jost-farum-x-change-3in...
The Bridge Vallombrosa Geldbörse blau               https://www.wardow.com/the-bridge-vallombrosa-...
The Bridge Vallombrosa Handtasch

In [177]:
def generate_random_url(): 
    random_index = random.randint(0, len(results))
    return results[random_index]

In [233]:
class Scraper: 
    def __init__(self, url): 
        self.url = url
    def get_product_info(self):
        driver = webdriver.Chrome(executable_path=r'C:\Users\Maria\chromedriver\chromedriver.exe')
        driver.get(self.url)
        product_url = self.url
        title = driver.find_element_by_xpath('//span[@class="product-name"]').text
        product_id = driver.find_element_by_xpath('//span[@itemprop="sku"]').text
        product_image_urls = [img.get_attribute('src') for img in driver.find_elements_by_xpath('//img[@class="gallery-image"]')]
        brand = driver.find_element_by_xpath('//span[@itemprop="brand"]').text
        price = driver.find_element_by_xpath('//span[contains(@id, "product-price")]').text
        prediscount_price = driver.find_element_by_xpath('//span[contains(@id, "old-price")]').text
        category = driver.find_element_by_xpath('//ul[@itemprop="breadcrumb"]//a').text
        product_description = driver.find_element_by_xpath('//dd[@class="tab-container tab-description current"]').text
        try: 
            in_stock = bool(driver.find_element_by_xpath('//p[@class="availability in-stock"]/span').text)
        except:
            print('Product out of stock!')
        finally: 
            driver.quit()
        print('Product Info for your requested page:')
        return pd.DataFrame.from_dict(locals(), orient='index', columns=['product_info']).drop(index=['self', 'driver'])

In [234]:
scraper = Scraper(generate_random_url())
scraper.get_product_info()

Product Info for your requested page:


,product_info
product_url,https://www.wardow.com/guess-dayane-umhangetas...
title,Dayane Umhängetasche Lederimitat rosa
product_id,HWSG7968700-BLS
product_image_urls,[https://www.wardow.com/images/1000x/guess-day...
brand,GUESS
price,"77,35 €"
prediscount_price,"119,00 €"
category,Katalog
product_description,Produktdetails\nabnehmbare Tasche\nschließt mi...
in_stock,True
